In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p5/train.csv
/kaggle/input/2022-ml-finalexam-p5/test.csv
/kaggle/input/2022-ml-finalexam-p5/sample_submit.csv


In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
submit = pd.read_csv('../input/2022-ml-finalexam-p5/sample_submit.csv')
train = pd.read_csv('../input/2022-ml-finalexam-p5/train.csv')
test = pd.read_csv('../input/2022-ml-finalexam-p5/test.csv')

In [4]:
# 필요 없는 열 제거
train = train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1)
test = test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1)

# 학습과 라벨 데이터 분리
trainX = train.drop(['Survived'], axis = 1)
trainY = train['Survived']

# Age

In [5]:
# imputer 사용하여 nan값 평균으로 채워주기
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
test['Age'] = imp_mean.fit_transform(np.array(test['Age']).reshape(-1, 1))
trainX['Age'] = imp_mean.transform(np.array(trainX['Age']).reshape(-1, 1))

# Sex

In [6]:
# 성별이 남자면 0, 여자면 1로 맵핑해주기

trainX['Sex'][trainX['Sex'] == 'male'] = 0
trainX['Sex'][trainX['Sex'] == 'female'] = 1

test['Sex'][test['Sex'] == 'male'] = 0
test['Sex'][test['Sex'] == 'female'] = 1

# 정수형으로 변환
trainX['Sex'] = trainX['Sex'].astype('int64')
test['Sex'] = test['Sex'].astype('int64')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

# Embarked, Cabin

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata = pd.concat([trainX, test])

# embarked 실수화시키기. 전체 데이터로 학습
le.fit(alldata['Embarked'])
trainX['Embarked'] = le.transform(trainX['Embarked'])
test['Embarked'] = le.transform(test['Embarked'])

In [8]:
# # nan값인 행 drop 시켜주기 -> cabin 열 자체를 drop 시킨 모델의 성능이 더 우수
# trainX['Cabin'] = trainX['Cabin'].dropna()
# le.fit(alldata['Cabin'])

# # 실수화시키기
# trainX['Cabin'] = le.transform(trainX['Cabin'])
# test['Cabin'] = le.transform(test['Cabin'])

In [9]:
# 표준화 시켜주기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
test = scaler.transform(test)

In [10]:
# randomforestclassifier로 예측
from sklearn.ensemble import RandomForestClassifier
# 인스턴스 생성
rfc = RandomForestClassifier(n_estimators = 50, random_state = seed)
# train data로 학습
rfc.fit(trainX, trainY)
# test 데이터로 예측
predict = rfc.predict(test)

In [11]:
# from sklearn.svm import SVC -> 성능 별로
# svc = SVC(random_state = seed)
# svc.fit(trainX, trainY)
# predict = svc.predict(test)

In [12]:
submit['Survived'] = predict
submit.to_csv('submission.csv', index = False)